In [2]:
import spacy
import requests

# Load the spaCy model for Named Entity Recognition (NER)
nlp = spacy.load("en_core_web_sm")

# Your Diffbot API token (Replace with your actual token)
DIFFBOT_TOKEN = "YOUR_DIFFBOT_API_KEY"
DIFFBOT_URL = "https://kg.diffbot.com/kg/v3/enhance"

def extract_named_entities(newsText):
    """
    Extracts named entities (persons, places, and products) from the news article.
    """
    doc = nlp(newsText)
    entities = set()

    for ent in doc.ents:
        if ent.label_ in ["PERSON", "GPE", "ORG", "PRODUCT"]:  # Filter relevant entity types
            entities.add(ent.text)

    return list(entities)

def query_diffbot(entity):
    """
    Queries the Diffbot Knowledge Graph for information on a given entity.
    """
    params = {
        "token": DIFFBOT_TOKEN,
        "query": entity,
        "size": 1  # Limit results to the most relevant match
    }
    response = requests.get(DIFFBOT_URL, params=params)

    if response.status_code == 200:
        data = response.json()
        if "data" in data and len(data["data"]) > 0:
            result = data["data"][0]  # Get the first (most relevant) result
            return {
                "name": result.get("name", entity),
                "type": result.get("type", "Unknown"),
                "description": result.get("description", "No description available"),
                "confidence": result.get("confidence", 0.0)  # Confidence score
            }
    return None

# Example news article
newsText = """
Elon Musk, the CEO of Tesla, announced the new Tesla Model X in California. 
Apple Inc. also introduced the iPhone 15 during an event in Cupertino.
"""

# Step 1: Extract entities from the news article
entities = extract_named_entities(newsText)

# Step 2: Query Diffbot for each entity
results = {}
for entity in entities:
    diffbot_data = query_diffbot(entity)
    if diffbot_data:
        results[entity] = diffbot_data

# Step 3: Display the results
for entity, info in results.items():
    print(f"\nEntity: {info['name']}")
    print(f" - Type: {info['type']}")
    print(f" - Description: {info['description']}")
    print(f" - Confidence: {info['confidence']}")


OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.